## Обзор доступных данных

В качестве данных мы будем использовать датасет, содержащий оценки пользователями товаров, представленных на платформе Amazon:

https://www.kaggle.com/c/recsystem-for-amazon-icl/overview

Обучающая выборка содержат следующие признаки: 
<ul>
<li><strong>userId</strong> - ID пользователя </li>
<li><strong>productId</strong> - ID продукта </li>
<li><strong>Rating</strong> - оценка соответствующего продукта от соответствующего пользователя </li>
</ul>

В тестовой выборке отсутствует признак Rating. 

Всего в выборке 42469 оценок, которые мы разбили на обучающую и тестовую части.  


# 1. Загрузить данные для обучения

## 1.1 Загружаем библиотеки 

Для решения задачи нам понадобятся уже знакомые: 
* библиотека **pandas** для работы с табличными данными (Excel, CSV и др.);
* библиотека **numpy** для обработки массивов чисел, с помощью которой можно быстро создавать массивы, производить над ними вычисления, сортировать и т.д.
* библиотека **matplotlib** для работы с двумерной и трехмерной графиков;

* библиотека **scikit-learn**, де-факто наиболее популярный, разносторонний, хорошо документированный и постоянно обогащающийся инструмент для построения моделей машинного обучения;

In [ ]:
import pandas as pd 
import numpy as np 

import matplotlib
%matplotlib inline

import sklearn

Отключим лишние предупреждения.

In [ ]:
import warnings; warnings.simplefilter('ignore')

Для создания алгоритмов рекомендательной системы, будем использовать библиотеку **surprise**.

Документация библиотеки доступна по ссылке http://surpriselib.com/.

In [ ]:
!pip install surprise

import surprise

## 1.2. Загрузим данные



In [ ]:
IN_COLAB = False

if IN_COLAB:
    !wget https://www.dropbox.com/s/y5glirn7hi88don/train.csv
    !wget https://www.dropbox.com/s/9ie39fr09lbfo52/test.csv
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('test.csv')
else:
    train_data = pd.read_csv('./data/train.csv')
    test_data = pd.read_csv('./data/test.csv')

Проверим, что данные загрузились правильно:

In [ ]:
train_data.head()

Проверим размерность данных:

In [ ]:
train_data.shape

## 1.3. Первичный анализ данных

Проверим типы в данных и количество NaN-ов:

In [ ]:
train_data.info()

Найдём минимальный и максимальные рейтинги:


In [ ]:
print('Minimum rating is: %d' %(train_data.Rating.min()))
print('Maximum rating is: %d' %(train_data.Rating.max()))

Посмотрим на распределение рейтингов:

In [ ]:
train_data.Rating.hist(figsize=(8, 6))

Пользователи часто ставят оценку 5.

**Упражнение:** сколько уникальных пользователей в выборке, а товаров?

In [ ]:
train_data.describe().T

In [ ]:
train_data.userId.nunique()


**Упражнение:** сколько оценок в среднем делает пользователь?

In [ ]:
train_data.groupby(['userId', 'Rating'])['userId'].count().head()

### 2. Обучить модель

### Алгоритм KNN. Коллаборативная фильтрация по схожести пользователей

Из библиотеки **surprise** импортируем специальный классы **Reader, Dataset**, которые позволят удобнее работать с данными.

In [ ]:
from surprise import Reader
from surprise import Dataset

In [ ]:
reader = Reader(sep=',', rating_scale=(1, 5))
train = Dataset.load_from_df(train_data, reader)
train = train.build_full_trainset()

В качестве baseline рассмотрим KNN, который использует схожесть **товаров** по косинусной метрике. 

In [ ]:
from surprise import KNNBasic
from surprise import accuracy

In [ ]:
algo = KNNBasic(k=3, sim_options={'name': 'cosine', 'user_based': False})
algo.fit(train)

In [ ]:
# run the trained model against the testset
test_pred = algo.test(test_data[['userId', 'productId', 'Rating']].values)

In [ ]:
def prepare_surprise_predictoin_for_kaggle(test_pred):
    key_list = []
    rate_list = [] 

    for i in range(0, len(test_pred)):
        key_list.append((test_pred[i][0], test_pred[i][1]))
        rate_list.append(test_pred[i][3])

    predictions = pd.DataFrame(np.transpose([key_list, rate_list]), columns=['Keys', 'Predicted rating'])
    return predictions

In [ ]:
pred_data = prepare_surprise_predictoin_for_kaggle(test_pred)

In [ ]:
pred_data.to_csv('predictions.csv', index=False)

# Идеи для дальнейшей работы: 

**1. Попробовать другие расстояния для модели** 

Для коллаборативной фильтрации можно попробовать различные расстояния: косинусное, корреляции Пирсона итд. 

**2. Попробовать другие модели**

Мы рассмотрели только одну модель user-based коллаборативной фильтрации. Рассмотрите item-based коллаборативную фильтрацию, алгоритм SVD.

**3. Подобрать параметры модели** 
Можно использовать метод GridSearch из библиотеки surprise. 
